# RSS: Economista

## 1. Import Libraries

In [46]:
import feedparser
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import googletrans
import datetime
from termcolor import colored
from IPython.display import display, Markdown, Latex, HTML
import re
from IPython.display import Image
import urllib

In [37]:
def get_only_text1(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    # Eliminar Última Hora
    rest = soup.find_all('p', attrs={'class' : "line-clamp_x2 ue-c-widget__article-headline"})[0].text
    index = text.find(rest)
    final_text = text[:index]
    return [final_text,soup.title.text]

def get_only_text(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    return [text,soup.title.text]

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [79]:
def economista_news(economista_entries,word_count=100):
    economista = {}
    for i in range(len(economista_entries)):
        noticia = {'newspaper':None,'title':None,'headline':None,'summarise':None,'date':None,'link':None,'text':None,'current_date':None,'img':None,'premium':None}
        # Periodico
        noticia['newspaper'] = find_between(s=economista_entries[i]['id'], first='www.el', last='.' )
        # Title
        noticia['title'] = economista_entries[i]['title']
        # Headline
        headline =  re.sub(pattern='&quot;',repl='"',string=economista_entries[i]['summary_detail']['value'])
        noticia['headline'] = headline
        # Date
        noticia['date'] = {'date1':economista_entries[i]['published'],'date2':economista_entries[i]['published_parsed']}
        # Link
        noticia['link'] = economista_entries[i]['id']
        # Text
        text = get_only_text(economista_entries[i]['id'])[0][len(headline)+1:]
        noticia['text'] = text
        # Summarise
        noticia['summarise'] = summarize(text, word_count=word_count)
        # Current Date
        noticia['current_date'] = datetime.datetime.now().timetuple()
        # Image
        noticia['img'] = None
        economista.update({'Noticia'+'_'+str("%02d")%(i+1):noticia})
    return economista

## 2. RSS Economista

In [80]:
url_economista = 'https://www.eleconomista.es/rss/rss-empresas.php'
economista = feedparser.parse(url_economista)
economista_entries = economista['entries']

print('Total Entradas Economista:',len(economista_entries))

Total Entradas Economista: 20


In [81]:
economista = economista_news(economista_entries,word_count=100)

In [82]:
yesterday = datetime.datetime.today() - datetime.timedelta(days=1)
before_yes = datetime.datetime.today() - datetime.timedelta(days=2)

for i in list(economista.keys()):
    if economista[i]['date']['date2'][2] in [datetime.datetime.today().day,yesterday.day,before_yes.day]:
        print(colored((economista[i]['newspaper']).upper(),'green',attrs=['bold']))
        print(colored(economista[i]['date']['date1'],'grey',attrs=['bold']))
        print(colored(economista[i]['title'],'magenta',attrs=['bold']))
        print(colored('Headline','blue'))
        print(economista[i]['headline'])
        print(colored('Summarise','blue'))
        
        if economista[i]['summarise'] != 'Premium Content':
            print(economista[i]['summarise'])
        else:
            print(colored(economista[i]['summarise'],'red'))
        print(colored('Text','blue'))
        print(economista[i]['text'])
        display(Markdown('<a href="'+economista[i]['link']+'">Link</a>'))
        print('\n')

ECONOMISTA
Tue, 07 Jul 2020 21:57:50 +0200
El Gobierno moviliza 1.773 millones más para ayudar al transporte
Headline
El Consejo de Ministros aprobó ayer el Real Decreto que el transporte de mercancías y pasajeros por carretera llevaba semanas esperando. Aunque los detalles publicados de las ayudas a "la reactivación económica" no cubren ni de lejos las medidas solicitadas por las patronales para asegurar la supervivencia del sector, sí que recogen una de las principales demandas: el aplazamiento de las cuotas de renting o leasing de los vehículos. El texto, presentado por el ministro José Luis Ábalos, también incluye un balón de oxigeno para Renfe y Seittsa y  medidas de alivio el sector portuario, elevando el presupuesto de este nuevo plan a los 1.773 millones.
Summarise
El texto, presentado por el ministro José Luis Ábalos, también incluye un balón de oxigeno para Renfe y Seittsa y  medidas de alivio el sector portuario, elevando el presupuesto de este nuevo plan a los 1.773 millone

<a href="http://www.eleconomista.es/empresas-finanzas/noticias/10652978/07/20/El-Gobierno-moviliza-1773-millones-mas-para-ayudar-al-transporte-.html">Link</a>



ECONOMISTA
Tue, 07 Jul 2020 21:36:35 +0200
Los puertos se 'vacían' de pasajeros y perderán el 15% de las mercancías
Headline
Aena ha reabierto la terminal 1 de Barajas, parte del Dique Longitudinal de El Prat y el módulo A de Palma ante el aumento de los vuelos internacionales. Así, mientras el número de operaciones aéreas diarias se ha triplicado desde que se reabrieron las fronteras, los puertos siguen cerrados a cal y canto a los cruceros.
Summarise
Al veto a los cruceros, publicado en el BOE el 27 de junio, se suma la decisión de Marruecos de mantener cerradas las fronteras con España, lo que obliga a cancelar la operación Paso del Estrecho; que el año pasado movió 3,34 millones de pasajeros y 760.215 vehículos, y el descenso de la demanda en los ferrys, que sí que operan.
De momento, Costa Cruceros tiene pensado fletar un buque en agosto desde Italia, pero todavía no tiene definido el itinerario porque la actividad sigue muy restringida para el sector, que el año pasado movió 10

<a href="http://www.eleconomista.es/empresas-finanzas/noticias/10652947/07/20/Los-puertos-se-vacian-de-pasajeros-y-perderan-el-15-de-las-mercancias-.html">Link</a>



ECONOMISTA
Tue, 07 Jul 2020 19:30:52 +0200
El mercado del plástico reciclado duplicará su valor hasta el año 2027
Headline
Las empresas apuestan por potenciar el mercado del plástico reciclado, cuyo valor crecerá a un ritmo del 6,8% anual, desde los 30.000 millones de euros registrados en 2019 a los 53.000 millones previstos para 2027, según el informe La ambición empresarial para avanzar hacia la nueva economía de plásticos, presentado este martes por Forética.
Summarise
Las empresas apuestan por potenciar el mercado del plástico reciclado, cuyo valor crecerá a un ritmo del 6,8% anual, desde los 30.000 millones de euros registrados en 2019 a los 53.000 millones previstos para 2027, según el informe La ambición empresarial para avanzar hacia la nueva economía de plásticos, presentado este martes por Forética.
Prevé también la introducción de un impuesto sobre los residuos plásticos, como el planteado por Bruselas, de 0,8 euros por kilo sobre los que no se reciclen, para fomentar la r

...

<a href="http://www.eleconomista.es/catalunya/noticias/10651991/07/20/COMSA-Industrial-se-adjudica-dos-proyectos-de-ingenieria-en-las-autopistas-de-Colombia-por-18-millones.html">Link</a>



ECONOMISTA
Tue, 07 Jul 2020 13:31:11 +0200
La empresa española Immunostep fabricará test de coronavirus del CSIC con un 98% de fiabilidad
Headline
La necesidad de contar con test de anticuerpos más fiables ha llevado al CSIC a diseñar un prototipo con el 98% de fiabilidad. Ahora, la empresa española Immunostep se encargará de su fabricación. El consejero delegado de la firma, Ricardo Jara, asegura que alcanzarán una producción de 14.000 kits diarios y que estarán en los centros sanitarios en cuestión de semanas.
Summarise
case 2 : jQuery(parrafos[1]).after(jQuery("div[class='b300 pub_mob251']")); break;
default: jQuery(parrafos[2]).after(jQuery("div[class='b300 pub_mob251']"));
"Con el nuevo test se solventan estos problemas ya que es una tecnología española, fabricada en España y que permitirá cubrir tanto la demanda nacional como la de otros países que lo puedan necesitar, y, además, cuenta con una fiabilidad del 98 por ciento", ha comentado.
"Con el nuevo test se solventan estos p

<a href="http://www.eleconomista.es/empresas-finanzas/noticias/10651674/07/20/La-empresa-espanola-Immunostep-fabricara-test-de-coronavirus-del-CSIC-con-un-98-de-fiabilidad.html">Link</a>



ECONOMISTA
Tue, 07 Jul 2020 12:53:43 +0200
BBVA, contrario a que PwC entregue los datos que unen a Francisco González con el espionaje a Ausbanc
Headline
BBVA ha recurrido el requerimiento que realizó el pasado lunes el juez del caso Villarejo a PwC para que, en un plazo de siete días, aportara a la causa todos los datos, documentos e información  a los que hayan tenido acceso que relacionen al expresidente del banco, Francisco González (FG), con el espionaje a la asociación de consumidores Ausbanc.
Summarise
Sin embargo, la entidad esgrime en su recurso, al que ha tenido acceso la agencia Efe, que el requerimiento del juez vulnera el derecho de defensa de BBVA y considera la imposibilidad de cumplir el plazo de una semana dado el volumen de información almacena.
PUES PARA IR PREGONANDO "SU ABSOLUTA COLABORACIÓN" EN ESTE TURBIO ASUNTO LA VERDAD ES QUE LO DISIMULAN BASTANTE BIEN.LO SIGUIENTE QUE VENDERA BBVA ES QUE ES "UN BANCO ETICO".
PARA IR PREGONANDO "SU ABSOLUTA COLABORACIÓN" EN 

<a href="http://www.eleconomista.es/empresas-finanzas/noticias/10651523/07/20/BBVA-contrario-a-que-se-entregue-los-datos-que-unan-a-Francisco-Gonzalez-con-el-espionaje-a-Ausbanc.html">Link</a>